# Convolutional Neural Networks

In this notebook, we will implement one of the first successful CNNs for handwritten digit recognition and adapt it to work with CIFAR10 data.

In [1]:
import time
import torch
import torch.nn as nn
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from tqdm.notebook import trange, tqdm
from torch.utils.tensorboard import SummaryWriter


%matplotlib widget

## LeNet5

The original network was meant for $28 \times 28$ single-channel images.
Our data is $32 \times 32 \times 3$. The first change is to update the number of input channels in the first layer.

In [18]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, 5), # nn.Conv2d(1, 6, 5) -> nn.Conv2d(3, 6, 5)
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.estimator = nn.Sequential(
            nn.Linear(400, 120), # nn.Linear(256, 120) -> nn.Linear(400, 120)
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.shape[0], -1)

        return self.estimator(x)

## Helper Functions

In [9]:
def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = transforms.functional.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
        
        
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

## Training and Evaluation Functions

In [21]:
def train_loop(dataloader, model, loss_fn, optimizer, logger=None):
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    pbar = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, (input, target) in pbar:
        
        input = input.cuda()
        target = target.cuda()

        output = model(input)
        loss = loss_fn(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        output = output.float()
        loss = loss.float()

        prec = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.shape[0])
        top1.update(prec.item(), input.shape[0])
        
        if i % print_frequency == 0:
            pbar.set_description("Epoch [%d]\t Loss %.2f\t Prec@1 %.3f (%.3f)" % (epoch, losses.avg, top1.val, top1.avg))
            if logger:
                logger.add_scalar("training loss",
                                  loss.item(),
                                  epoch * len(dataloader) + i)
           
        
def val_loop(dataloader, model, loss_fn, logger=None):
    losses = AverageMeter()
    top1 = AverageMeter()

    model.eval()

    pbar = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, (input, target) in pbar:
        
        input = input.cuda()
        target = target.cuda()

        output = model(input)
        loss = loss_fn(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        output = output.float()
        loss = loss.float()
        
        prec = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.shape[0])
        top1.update(prec.item(), input.shape[0])

        if i % print_frequency == 0:
            pbar.set_description("Epoch [%d]\t Loss %.2f\t Prec@1 %.3f (%.3f)" % (epoch, losses.avg, top1.val, top1.avg))
            if logger:
                logger.add_scalar("validation loss",
                                  loss.item(),
                                  epoch * len(dataloader) + i)
    
    if logger:
        logger.add_scalar("validation accuracy",
                          top1.avg,
                          epoch)
            

def test_loop(dataloader, model, loss_fn):
    losses = AverageMeter()
    top1 = AverageMeter()
    
    model.eval()
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, (input, target) in pbar:
        
        input = input.cuda()
        target = target.cuda()

        output = model(input)
        loss = criterion(output, target)

        output = output.float()
        loss = loss.float()

        prec = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.shape[0])
        top1.update(prec.item(), input.shape[0])

    # Print result
    print(f"Average Loss: {losses.avg:>8f}\nAccuracy: {top1.avg}\n")

# Model Hyperparameters and Initialization

In [32]:
# Model Parameters
batch_size = 256
learning_rate = 1e-3
epochs = 100
print_frequency = 100

model = LeNet5()
model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

## Dataset Preparation

In [31]:
root_path = "/home/alex/Data/CIFAR10"
# root_path = "/Users/ajd/Data/CIFAR10"

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

dataset = torchvision.datasets.CIFAR10(root_path, train=True, transform=transforms.Compose([
    transforms.RandAugment(),
    transforms.ToTensor(),
    normalize
]), download=True)

dataset_size = len(dataset)
train_size = int(dataset_size * .95)
val_size = dataset_size - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size, shuffle=True,
    num_workers=8, pin_memory=False)


val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size, shuffle=False,
    num_workers=8, pin_memory=False)


test_dataloader = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR10(root_path, train=False, transform=transforms.Compose([                                                                     
        transforms.ToTensor(),
        normalize
    ])),
    batch_size=batch_size, shuffle=False,
    num_workers=8, pin_memory=False)

Files already downloaded and verified


## Train model

In [33]:
logger = SummaryWriter("runs/lenet5_basic")

for epoch in range(epochs):
    train_loop(train_dataloader, model, criterion, optimizer, logger)
    val_loop(val_dataloader, model, criterion, logger)

  0%|          | 0/186 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/186 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/186 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/186 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/186 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [30]:
test_loop(test_dataloader, model, criterion)

  0%|          | 0/40 [00:00<?, ?it/s]

Average Loss: 2.722911
Accuracy: 54.57

